In [13]:
import numpy as np
import os
import time

1. Генерация точек

In [21]:
def generate_synthetic_cloud(n: int = 100_000, low: float = 0.0, high: float = 100.0, seed: int = 42) -> np.ndarray:
    """
    Генерация синтетического облака точек размера n.
    Координаты равномерно распределены в кубе [low; high]^3.
    """
    rng = np.random.default_rng(seed)
    return rng.uniform(low, high, size=(n, 3))

In [22]:
def save_xyz(path: str, points: np.ndarray) -> None:
    """
    Сохранение облака в .xyz (3 колонки: X Y Z).
    """
    np.savetxt(path, points, fmt="%.6f")

In [23]:
out_dir = "task2b_outputs"
os.makedirs(out_dir, exist_ok=True)

points = generate_synthetic_cloud(n=100_000, low=0.0, high=100.0, seed=42)
save_xyz(os.path.join(out_dir, "synthetic_cloud.xyz"), points)

print("Сгенерировано синтетическое облако:", points.shape)

Сгенерировано синтетическое облако: (100000, 3)


2. Выделение области по координатным границам (Bounding Box)

In [7]:
def filter_bbox(points: np.ndarray,
                xmin: float, xmax: float,
                ymin: float, ymax: float,
                zmin: float, zmax: float) -> np.ndarray:
    """
    Фильтр A: Bounding Box (прямоугольный параллелепипед).
    Оставляет точки, которые попадают в диапазоны по X, Y, Z.
    """
    mask = (
        (points[:, 0] >= xmin) & (points[:, 0] <= xmax) &
        (points[:, 1] >= ymin) & (points[:, 1] <= ymax) &
        (points[:, 2] >= zmin) & (points[:, 2] <= zmax)
    )
    return points[mask]

In [14]:
bbox = dict(xmin=20, xmax=50, ymin=30, ymax=70, zmin=10, zmax=40)

t0 = time.perf_counter()
pts_bbox = filter_bbox(points, **bbox)
t1 = time.perf_counter()

save_xyz(os.path.join(out_dir, "bbox_filtered.xyz"), pts_bbox)

print("BBox:", bbox)
print("  Получено точек:", pts_bbox.shape[0])
print(f"  Время: {t1 - t0:.6f} сек")

BBox: {'xmin': 20, 'xmax': 50, 'ymin': 30, 'ymax': 70, 'zmin': 10, 'zmax': 40}
  Получено точек: 3532
  Время: 0.001216 сек


3. Выделение по высоте (например, только "высокие" точки)

In [15]:
def filter_height(points: np.ndarray, zmin: float | None = None, zmax: float | None = None) -> np.ndarray:
    """
    Фильтр B: По высоте (по Z).
    Можно задать нижний порог zmin и/или верхний порог zmax.
    """
    mask = np.ones(points.shape[0], dtype=bool)
    if zmin is not None:
        mask &= (points[:, 2] >= zmin)
    if zmax is not None:
        mask &= (points[:, 2] <= zmax)
    return points[mask]

In [16]:
zmin = 80.0 

t0 = time.perf_counter()
pts_high = filter_height(points, zmin=zmin)
t1 = time.perf_counter()

save_xyz(os.path.join(out_dir, "high_points_zmin80.xyz"), pts_high)

print(f"Height filter: Z >= {zmin}")
print("  Получено точек:", pts_high.shape[0])
print(f"  Время: {t1 - t0:.6f} сек")


Height filter: Z >= 80.0
  Получено точек: 20016
  Время: 0.001787 сек


4. Выделение по расстоянию до заданной точки

In [18]:
def filter_distance(points: np.ndarray, center: np.ndarray, radius: float) -> np.ndarray:
    """
    Фильтр C: По расстоянию до заданной точки (сфера).
    Оставляет точки на расстоянии <= radius от center.
    """
    center = np.asarray(center, dtype=float)
    d = np.linalg.norm(points - center, axis=1)
    return points[d <= radius]


In [20]:
center = np.array([50.0, 50.0, 50.0])
radius = 15.0

t0 = time.perf_counter()
pts_near = filter_distance(points, center=center, radius=radius)
t1 = time.perf_counter()

save_xyz(os.path.join(out_dir, "near_center_R15.xyz"), pts_near)

print("Distance filter:")
print("  center =", center)
print("  radius =", radius)
print("  Получено точек:", pts_near.shape[0])
print(f"  Время: {t1 - t0:.6f} сек")

Distance filter:
  center = [50. 50. 50.]
  radius = 15.0
  Получено точек: 1382
  Время: 0.008404 сек
